# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jul 22 2022 1:22PM,242555,20,7729858-FLS-BO,"ACI Healthcare USA, Inc.",Released
1,Jul 22 2022 1:22PM,242555,20,7670931-BO-FLS,"ACI Healthcare USA, Inc.",Released
2,Jul 22 2022 1:22PM,242555,20,7812405-FLS,"ACI Healthcare USA, Inc.",Released
3,Jul 22 2022 1:22PM,242555,20,7729874-FLS-BO1-FLS,"ACI Healthcare USA, Inc.",Released
4,Jul 22 2022 1:22PM,242555,20,7903098-FLS-BO-FLS,"ACI Healthcare USA, Inc.",Released
5,Jul 22 2022 1:22PM,242555,20,7965692-FLS,"ACI Healthcare USA, Inc.",Released
6,Jul 22 2022 1:22PM,242555,20,7996544-FLS,"ACI Healthcare USA, Inc.",Released
7,Jul 22 2022 1:22PM,242555,20,8091856,"ACI Healthcare USA, Inc.",Released
8,Jul 22 2022 1:22PM,242555,20,8070448-BO,"ACI Healthcare USA, Inc.",Released
9,Jul 22 2022 1:22PM,242555,20,8070467-BO,"ACI Healthcare USA, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
20,242553,Released,3
21,242554,Executing,6
22,242554,Released,1
23,242555,Cancelled,1
24,242555,Released,53


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Cancelled,Executing,Released
id,,,
242551,NaN,NaN,1.0
242552,NaN,NaN,1.0
242553,NaN,1.0,3.0
242554,NaN,6.0,1.0
242555,1.0,NaN,53.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Cancelled,Executing,Released
id,,,
242450,0.0,0.0,4.0
242464,0.0,1.0,0.0
242467,0.0,0.0,1.0
242498,0.0,0.0,1.0
242505,0.0,0.0,15.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,Cancelled,Executing,Released
id,,,
242450,0.0,0,4
242464,0.0,1,0
242467,0.0,0,1
242498,0.0,0,1
242505,0.0,0,15


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Cancelled,Executing,Released
0,242450,0.0,0,4
1,242464,0.0,1,0
2,242467,0.0,0,1
3,242498,0.0,0,1
4,242505,0.0,0,15


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

Column Completed does not exist yet


ShipmentStatus,id,Cancelled,Executing,Released
0,242450,0.0,,4
1,242464,0.0,1,
2,242467,0.0,,1
3,242498,0.0,,1
4,242505,0.0,,15


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id             int64
Cancelled    float64
Executing     object
Released      object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jul 22 2022 1:22PM,242555,20,"ACI Healthcare USA, Inc."
54,Jul 22 2022 1:15PM,242554,19,Dexcel Pharma Technologies Ltd. DBA WDSrx
61,Jul 22 2022 1:12PM,242553,19,"AdvaGen Pharma, Ltd"
65,Jul 22 2022 12:59PM,242552,10,Beach Patient Assistance
66,Jul 22 2022 12:58PM,242551,12,Hush Hush
67,Jul 22 2022 12:50PM,242550,18,"Ayrton Saunders, Inc."
68,Jul 22 2022 12:43PM,242549,10,"ClearSpec, LLC"
71,Jul 22 2022 12:38PM,242547,18,"Ayrton Saunders, Inc."
72,Jul 22 2022 12:25PM,242545,10,"Methapharm, Inc."
78,Jul 22 2022 12:00PM,242540,10,Eywa Pharma Inc.


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Cancelled,Executing,Released
0,Jul 22 2022 1:22PM,242555,20,"ACI Healthcare USA, Inc.",1.0,,53
1,Jul 22 2022 1:15PM,242554,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,0.0,6,1
2,Jul 22 2022 1:12PM,242553,19,"AdvaGen Pharma, Ltd",0.0,1,3
3,Jul 22 2022 12:59PM,242552,10,Beach Patient Assistance,0.0,,1
4,Jul 22 2022 12:58PM,242551,12,Hush Hush,0.0,,1
5,Jul 22 2022 12:50PM,242550,18,"Ayrton Saunders, Inc.",0.0,,1
6,Jul 22 2022 12:43PM,242549,10,"ClearSpec, LLC",0.0,,3
7,Jul 22 2022 12:38PM,242547,18,"Ayrton Saunders, Inc.",0.0,,1
8,Jul 22 2022 12:25PM,242545,10,"Methapharm, Inc.",0.0,,6
9,Jul 22 2022 12:00PM,242540,10,Eywa Pharma Inc.,0.0,8,5


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 22 2022 1:22PM,242555,20,"ACI Healthcare USA, Inc.",53,
1,Jul 22 2022 1:15PM,242554,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,1,6
2,Jul 22 2022 1:12PM,242553,19,"AdvaGen Pharma, Ltd",3,1
3,Jul 22 2022 12:59PM,242552,10,Beach Patient Assistance,1,
4,Jul 22 2022 12:58PM,242551,12,Hush Hush,1,
5,Jul 22 2022 12:50PM,242550,18,"Ayrton Saunders, Inc.",1,
6,Jul 22 2022 12:43PM,242549,10,"ClearSpec, LLC",3,
7,Jul 22 2022 12:38PM,242547,18,"Ayrton Saunders, Inc.",1,
8,Jul 22 2022 12:25PM,242545,10,"Methapharm, Inc.",6,
9,Jul 22 2022 12:00PM,242540,10,Eywa Pharma Inc.,5,8


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 22 2022 1:22PM,242555,20,"ACI Healthcare USA, Inc.",53,
1,Jul 22 2022 1:15PM,242554,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,1,6
2,Jul 22 2022 1:12PM,242553,19,"AdvaGen Pharma, Ltd",3,1
3,Jul 22 2022 12:59PM,242552,10,Beach Patient Assistance,1,
4,Jul 22 2022 12:58PM,242551,12,Hush Hush,1,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

Column Completed does not exist yet


C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
C:\Users\jmeza.WOODFIELD\.conda\envs\PythonData\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


,Date,id,Warehouse,Customer,Released,Executing
0,Jul 22 2022 1:22PM,242555,20,"ACI Healthcare USA, Inc.",53.0,NaN
1,Jul 22 2022 1:15PM,242554,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,1.0,6.0
2,Jul 22 2022 1:12PM,242553,19,"AdvaGen Pharma, Ltd",3.0,1.0
3,Jul 22 2022 12:59PM,242552,10,Beach Patient Assistance,1.0,NaN
4,Jul 22 2022 12:58PM,242551,12,Hush Hush,1.0,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing
0,Jul 22 2022 1:22PM,242555,20,"ACI Healthcare USA, Inc.",53.0,0.0
1,Jul 22 2022 1:15PM,242554,19,Dexcel Pharma Technologies Ltd. DBA WDSrx,1.0,6.0
2,Jul 22 2022 1:12PM,242553,19,"AdvaGen Pharma, Ltd",3.0,1.0
3,Jul 22 2022 12:59PM,242552,10,Beach Patient Assistance,1.0,0.0
4,Jul 22 2022 12:58PM,242551,12,Hush Hush,1.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing
Warehouse,,,
10,1940224,36.0,8.0
12,485088,1.0,1.0
16,242498,1.0,0.0
18,485097,2.0,0.0
19,485107,4.0,7.0
20,970069,70.0,0.0
21,485003,1.0,1.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing
0,10,1940224,36.0,8.0
1,12,485088,1.0,1.0
2,16,242498,1.0,0.0
3,18,485097,2.0,0.0
4,19,485107,4.0,7.0
5,20,970069,70.0,0.0
6,21,485003,1.0,1.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing
0,10,36.0,8.0
1,12,1.0,1.0
2,16,1.0,0.0
3,18,2.0,0.0
4,19,4.0,7.0
5,20,70.0,0.0
6,21,1.0,1.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,36.0
1,12,Released,1.0
2,16,Released,1.0
3,18,Released,2.0
4,19,Released,4.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,12,16,18,19,20,21
Status,,,,,,,
Executing,8.0,1.0,0.0,0.0,7.0,0.0,1.0
Released,36.0,1.0,1.0,2.0,4.0,70.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,12,16,18,19,20,21
0,Executing,8.0,1.0,0.0,0.0,7.0,0.0,1.0
1,Released,36.0,1.0,1.0,2.0,4.0,70.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,12,16,18,19,20,21
0,Executing,8.0,1.0,0.0,0.0,7.0,0.0,1.0
1,Released,36.0,1.0,1.0,2.0,4.0,70.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()